<div style="direction:rtl;text-align:center"><img src="https://dl.mohammadkh.ir/logo.png" alt="Mohammadkh.ir" style="width: 250px;"/></div>
<h1><div style="direction:rtl;text-align:center">MediaPipe</div></h1>

In [1]:
import cv2
import mediapipe as mp
import math
import numpy as np
from ctypes import cast, POINTER
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume

In [2]:
# detector
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(
    model_complexity=0,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5)


mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

In [3]:
# find and read speaker
devices = AudioUtilities.GetSpeakers()
interface = devices.Activate(IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
volume = cast(interface, POINTER(IAudioEndpointVolume))

# get vol rage pc
volRange = volume.GetVolumeRange()
minVol , maxVol , volBar, volPer= volRange[0] , volRange[1], 400, 0

In [4]:
# size frame
wCam, hCam = 640, 480
cam = cv2.VideoCapture(0)
cam.set(3,wCam)
cam.set(4,hCam)

True

In [5]:


while cam.isOpened():
    # read frame
    success, image = cam.read()
    
    # preproccess
    image = cv2.flip(image, 1)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = hands.process(image)
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    
    
    # check funid hand
    if results.multi_hand_landmarks:
        # drew landmarks hands
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(
                image,
                hand_landmarks,
                mp_hands.HAND_CONNECTIONS,
                mp_drawing_styles.get_default_hand_landmarks_style(),
                mp_drawing_styles.get_default_hand_connections_style()
                )

    # list loc finger     
    lmList = []
    # check funid hand
    if results.multi_hand_landmarks:
        myHand = results.multi_hand_landmarks[0] # 0 --> hand 1
        for id, lm in enumerate(myHand.landmark):
            h, w, c = image.shape
            cx, cy = int(lm.x * w), int(lm.y * h)
            lmList.append([id, cx, cy])          

    # check funid finger
    if len(lmList) != 0:
        # cal target finger
        x1, y1 = lmList[4][1], lmList[4][2]  # 4 - 8 pointer hand
        x2, y2 = lmList[8][1], lmList[8][2]

        # drew 
        cv2.circle(image, (x1,y1),15,(255,255,255))  
        cv2.circle(image, (x2,y2),15,(255,255,255))   
        cv2.line(image,(x1,y1),(x2,y2),(0,255,0),3)
        length = math.hypot(x2-x1,y2-y1)
        if length < 50 or length > 220:
            cv2.line(image,(x1,y1),(x2,y2),(0,0,255),3)
        
        # set volume
        ## conv len to range
        vol = np.interp(length, [50, 220], [minVol, maxVol]) # convert lenth to volume rang
        volume.SetMasterVolumeLevel(vol, None)
        
        # drew Volume Bar
        volBar = np.interp(length, [50, 220], [400, 150])
        volPer = np.interp(length, [50, 220], [0, 100])
        cv2.rectangle(image, (50, 150), (85, 400), (0, 0, 0), 3)
        cv2.rectangle(image, (50, int(volBar)), (85, 400), (0, 0, 0), -1)
        cv2.putText(image, f'{int(volPer)} %', (40, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 0, 0), 3)

    cv2.imshow('handDetector', image) 
    if cv2.waitKey(1) == ord("q"):
        break
cam.release()
cv2.destroyAllWindows()

<div class="alert alert-block alert-info">
<div style="direction:rtl;text-align:left"><strong>MediaPipe</strong><br>MohammadReza <strong>Khajedaloi</strong><br><br>
</div>
<div style="direction:rtl;text-align:right">
<a href="http://mohammadkh.ir/">WebSite</a> - <a href="https://github.com/khajedaloi/">GitHub</a> - <a href="https://www.linkedin.com/in/mohammad-kh/">Linkedin</a>
</div>
</div>